# Custom DeBERTA V3

> Model Definitions goes here

In [ ]:
#| default_exp models

In [ ]:
#| export
import transformers
import torch
import torch.nn as nn

from transformers import AutoModel, AutoTokenizer, AutoConfig

In [ ]:
from en_grammar_checker.config import Config

In [ ]:
#| hide
cnfg = Config()

In [ ]:
# checking model config
AutoConfig.from_pretrained(cnfg.base_model_name)

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.40.1",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

In [ ]:
# checking tokenizer
my_tokenizer = AutoTokenizer.from_pretrained(cnfg.base_model_name)

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
a = my_tokenizer('I am done.')
b = my_tokenizer('Thanks! I am done')
c = my_tokenizer('I am done!')
d = my_tokenizer('[CLS]I am done!')

print(a)
print(b)
print(c)
print(d)

{'input_ids': [1, 273, 481, 619, 260, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
{'input_ids': [1, 1195, 300, 273, 481, 619, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [1, 273, 481, 619, 300, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
{'input_ids': [1, 1, 273, 481, 619, 300, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [ ]:
# Note: Tokenizer adds a starting token [CLS] and end of sentence token on its own 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()